# Logistic Regression
Implementation of logistic regression for binary class.

## Imports

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
%matplotlib inline

## Dataset

## Model

In [ ]:
class LogisticRegression(torch.nn.modules):
    def __init__(self, num_features):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(num_features, 1)
        # change random weigths to zero
        self.linear.weight.detach().zero_()
        self.linear.bias.detach().zeor_()
    
    def forward(self, x):
        netinputs = self.linear(x)
        output = torch.sigmoid(netinputs)
        return output
        
        